---
layout: default
categories: logisticRegression
title: "Neural network - Implementations"
permalink: /ML17.5/
order: 17.5
comments: true
---

In [5]:
%reset -f
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from types import SimpleNamespace

Once again we are using [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) as in <a href="page:ML7.5">ML-7.5</a>. 

This time around, instead of getting them as PIL images to then manually convert them to tensors, we are exploiting the `transform` argument of the datasets objects [[Datasets](https://pytorch.org/vision/stable/datasets.html)]. 

We are passing the `transforms.toTensor()` method that converts a PIL Image or `numpy.ndarray` (H x W x C) in the range [0, 255] to a `torch.FloatTensor` of shape (C x H x W) in the range [0.0, 1.0] [[Transforms](https://pytorch.org/vision/stable/transforms.html)].

In [13]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [14]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [15]:
len(train_dataset)

60000

The MNIST training set consists of 60000 images but we don't want to train the algorithm as a single batch. Instead, as discussed in <a href="page:ML29">ML-29</a> we are going to train the algorithm on mini-batches of the original training set. 

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [5]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [9]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.8086466789245605. Accuracy: 86.12999725341797
Iteration: 1000. Loss: 0.35314568877220154. Accuracy: 89.56999969482422
Iteration: 1500. Loss: 0.48396095633506775. Accuracy: 90.55999755859375
Iteration: 2000. Loss: 0.2627964913845062. Accuracy: 91.33000183105469
Iteration: 2500. Loss: 0.35829463601112366. Accuracy: 91.86000061035156
Iteration: 3000. Loss: 0.27417317032814026. Accuracy: 92.16999816894531
